In [15]:
%matplotlib inline
import os
import pandas as pd
import json
import folium
import numpy as np
import matplotlib.pyplot as plt
import datetime, time
import seaborn as sns
import sklearn
from sklearn import tree
from sklearn.cross_validation import KFold
from sklearn.model_selection import train_test_split, KFold, cross_val_predict, cross_val_score, GridSearchCV
from sklearn import datasets, linear_model
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn import svm

from scipy import stats

/Users/yev/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [510]:
def select_rows(df,from_col,row_list):
    df = df.dropna()
    df = df[df[from_col].str.contains(row_list)].set_index(from_col)
    #df = df.loc[:,'Median']
    return df

In [672]:
data_dir = 'data/wrangled_data/'
swiss_foreign = pd.read_csv('data/foreignswiss.csv')

In [679]:
def select_rows(df,from_col,row_list):
    df = df.dropna()
    df = df[df[from_col].str.contains(row_list)].set_index(from_col)
    #df = df.loc[:,'Median']
    return df

def get_income(wage_df):
    swiss = float(wage_df.loc[wage_df.index.str.contains('Swiss')].values[0][0])/float(wage_df.loc[wage_df.index.str.contains('TOTAL')].values[0][0])
    foreign = float(wage_df.loc[wage_df.index.str.contains('Foreigners')].values[0][0])/float(wage_df.loc[wage_df.index.str.contains('TOTAL')].values[0][0])
    return swiss, foreign

def get_med_income_df():
    data = ['central','east','lake_ge','mittelland','north','ti','zh']
    use_rows = 'TOTAL|Swiss|Foreigners'
    med_income = pd.read_csv('med_incomes.csv')
    med_income16 = med_income.loc[:,med_income.columns.isin(['Name','2016','Kantonsnr', 'region'])]
    wages_16 = []
    for region in data:
        wages_16.append(pd.read_csv(data_dir+'wages2016'+region+'.csv', usecols=['Unnamed: 0.1',' Median']).dropna(how='all').rename(columns={'Unnamed: 0.1': 'Resident_'+region}))
    for i in range(len(wages_16)):
        wages_16[i] = select_rows(wages_16[i],'Resident_'+data[i], use_rows)
    swiss = []
    foreign = []
    for region in wages_16:
        x, y = get_income(region)
        swiss.append(x)
        foreign.append(y)
    med_income16['Swiss_income'] = np.nan
    med_income16['Foreign_income'] = np.nan
    for region, sw, fo in zip(data,swiss,foreign):
        med_income16['Swiss_income'] = med_income16.apply(lambda r: (r['2016'] * sw * 100) if r['region'] == region else r['Swiss_income'], axis=1)
        med_income16['Foreign_income'] = med_income16.apply(lambda r: (r['2016'] * fo * 100) if r['region'] == region else r['Foreign_income'], axis=1)
    return med_income16

def get_household_df():
    df = pd.read_csv(data_dir + 'hh_sizes2016.csv')
    cols = ['1-person households','2-person households','3-person households','4-person households','5-person households','6-person or larger households','Gemeindecode']
    df['Unnamed: 0.1'] = df['Unnamed: 0.1'].str.replace('000','')
    df['Unnamed: 0.1'] = df['Unnamed: 0.1'].str.replace('00','')
    df['Unnamed: 0.1'] = df['Unnamed: 0.1'].str.replace('0','')
    df['Gemeindecode'] = df['Unnamed: 0.1'].str.replace(r'\D+', '')
    df = df[cols]
    if df.iloc[0][0] > 1000000:
        df = df.drop(0)
    return df

In [680]:
med_income_df = get_med_income_df()
hh_size_df = get_household_df()

/Users/yev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/yev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/yev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

In [683]:
df_transposed = hh_size_df.T.reset_index().rename(columns={'index': 'household'})

In [684]:
df_transposed

,household,1,2,3,4,5,6,7,8,9,...,2278,2279,2280,2281,2282,2283,2284,2285,2286,2287
0,1-person households,235,1689,626,398,399,125,230,79,486,...,208,46,21,1436,16,60,180,206,144,156
1,2-person households,548,3434,1592,1066,1076,306,558,162,1450,...,452,94,54,1938,62,174,356,404,286,342
2,3-person households,348,1935,924,597,633,129,378,102,840,...,291,84,45,1038,24,75,177,198,171,177
3,4-person households,544,2572,1632,960,1060,296,640,164,1264,...,432,72,32,1316,36,128,280,224,172,304
4,5-person households,190,1280,485,340,335,165,230,95,550,...,195,30,30,610,5,65,150,170,105,110
5,6-person or larger households,104,674,159,186,169,82,121,20,183,...,93,66,0,263,12,50,61,82,49,121
6,Gemeindecode,1,2,3,4,5,6,7,8,9,...,679,6792,6793,68,683,686,687,688,689,681


In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [107]:
rfc = RandomForestClassifier() 
estimators = [10,20,25,30,35,40,50,55,60,65,70]
depth = [1,2,4,6,8,10,12,14,16,18,20,22,24]

# Use a grid over parameters of interest
param_grid = { 
           "n_estimators" : estimators,
           "max_depth" : depth
            }

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)
print(CV_rfc.best_params_)

{'max_depth': 8, 'n_estimators': 35}


In [124]:
#Taking the best parameters from the Random Forest Classifier and Grid Search
CV_rfc_est = RandomForestClassifier(
                n_estimators= 35, 
                max_depth=8
                )
CV_rfc_est.fit(X_train, y_train)
prediction = CV_rfc_est.predict(test_data)
print(prediction)

[4]


In [123]:
columns = ['Region','Swiss','Income']
input_data = [(int(1), int(1), int(7000))]
test_data = pd.DataFrame(input_data, columns = columns)

In [119]:
test_data

,Region,Swiss,Income
0,1,1,4000
